In [1]:
import tensorflow as tf
import numpy as np
import unicodedata
import re

数据是一个元组列表，其中每个元组包含一个英语句子和一个法语句子。

In [2]:
raw_data = (
    ('What a ridiculous concept!', 'Quel concept ridicule !'),
    ('Your idea is not entirely crazy.', "Votre idée n'est pas complètement folle."),
    ("A man's worth lies in what he is.", "La valeur d'un homme réside dans ce qu'il est."),
    ('What he did is very wrong.', "Ce qu'il a fait est très mal."),
    ("All three of you need to do that.", "Vous avez besoin de faire cela, tous les trois."),
    ("Are you giving me another chance?", "Me donnez-vous une autre chance ?"),
    ("Both Tom and Mary work as models.", "Tom et Mary travaillent tous les deux comme mannequins."),
    ("Can I have a few minutes, please?", "Puis-je avoir quelques minutes, je vous prie ?"),
    ("Could you close the door, please?", "Pourriez-vous fermer la porte, s'il vous plaît ?"),
    ("Did you plant pumpkins this year?", "Cette année, avez-vous planté des citrouilles ?"),
    ("Do you ever study in the library?", "Est-ce que vous étudiez à la bibliothèque des fois ?"),
    ("Don't be deceived by appearances.", "Ne vous laissez pas abuser par les apparences."),
    ("Excuse me. Can you speak English?", "Je vous prie de m'excuser ! Savez-vous parler anglais ?"),
    ("Few people know the true meaning.", "Peu de gens savent ce que cela veut réellement dire."),
    ("Germany produced many scientists.", "L'Allemagne a produit beaucoup de scientifiques."),
    ("Guess whose birthday it is today.", "Devine de qui c'est l'anniversaire, aujourd'hui !"),
    ("He acted like he owned the place.", "Il s'est comporté comme s'il possédait l'endroit."),
    ("Honesty will pay in the long run.", "L'honnêteté paye à la longue."),
    ("How do we know this isn't a trap?", "Comment savez-vous qu'il ne s'agit pas d'un piège ?"),
    ("I can't believe you're giving up.", "Je n'arrive pas à croire que vous abandonniez."),
)

### 数据预处理
我们需要稍微清理原始数据。 这种任务通常包括规范化字符串，过滤不需要的字符，在标点符号前添加空格等。大多数时候，需要两个函数，如下所示：

In [3]:
def unicode_to_ascii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn')


def normalize_string(s):
    s = unicode_to_ascii(s)
    s = re.sub(r'([!.?])', r' \1', s)
    s = re.sub(r'[^a-zA-Z.!?]+', r' ', s)
    s = re.sub(r'\s+', r' ', s)
    return s

我们现在将数据拆分为两个单独的列表，每个列表包含自己的句子。 然后我们将应用上面的函数并添加两个特殊标记：<start>和<end>：

In [4]:
raw_data_en, raw_data_fr = list(zip(*raw_data))
raw_data_en, raw_data_fr = list(raw_data_en), list(raw_data_fr)

raw_data_en = [normalize_string(data) for data in raw_data_en]
raw_data_fr_in = ['<start> ' + normalize_string(data) for data in raw_data_fr]
raw_data_fr_out = [normalize_string(data) + ' <end>' for data in raw_data_fr]

In [57]:
print(raw_data_en)
print(raw_data_fr_in)
print(raw_data_fr_out)

['What a ridiculous concept !', 'Your idea is not entirely crazy .', 'A man s worth lies in what he is .', 'What he did is very wrong .', 'All three of you need to do that .', 'Are you giving me another chance ?', 'Both Tom and Mary work as models .', 'Can I have a few minutes please ?', 'Could you close the door please ?', 'Did you plant pumpkins this year ?', 'Do you ever study in the library ?', 'Don t be deceived by appearances .', 'Excuse me . Can you speak English ?', 'Few people know the true meaning .', 'Germany produced many scientists .', 'Guess whose birthday it is today .', 'He acted like he owned the place .', 'Honesty will pay in the long run .', 'How do we know this isn t a trap ?', 'I can t believe you re giving up .']
['<start> Quel concept ridicule !', '<start> Votre idee n est pas completement folle .', '<start> La valeur d un homme reside dans ce qu il est .', '<start> Ce qu il a fait est tres mal .', '<start> Vous avez besoin de faire cela tous les trois .', '<star

Seq2Seq模型由两个网络组成：编码器和解码器。 编码器位于左侧，仅需要源语言的序列作为输入。

另一方面，解码器需要两种版本的目标语言序列，一种用于输入，一种用于目标（Loss计算）。 解码器本身通常被称为语言模型。

从实验中，我还发现最好不要将<start>和<end>标记添加到源序列中。 这样做会使模型，尤其是后来的注意机制混淆，因为所有序列都以相同的标记开头。

接下来，让我们看看如何标记数据，即将原始字符串转换为整数序列。 我们将使用Keras的文本标记化实用程序类：

In [5]:
en_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')

注意filters参数。 默认情况下，Keras的Tokenizer会删除所有标点符号，这不是我们想要的。 由于我们已经过滤掉了标点符号（！？），我们可以在此处将过滤器设置为空白。

标记化的关键部分是词汇。 Keras的Tokenizer类附带了一些方法。 由于我们的数据包含原始字符串，因此我们将使用名为fit_on_texts的数据。

In [9]:
en_tokenizer.fit_on_texts(raw_data_en)

In [10]:
print(en_tokenizer.word_index)

{'.': 1, 'you': 2, '?': 3, 'the': 4, 'a': 5, 'is': 6, 'he': 7, 'what': 8, 'in': 9, 'do': 10, 'can': 11, 't': 12, 'did': 13, 'giving': 14, 'me': 15, 'i': 16, 'few': 17, 'please': 18, 'this': 19, 'know': 20, 'ridiculous': 21, 'concept': 22, '!': 23, 'your': 24, 'idea': 25, 'not': 26, 'entirely': 27, 'crazy': 28, 'man': 29, 's': 30, 'worth': 31, 'lies': 32, 'very': 33, 'wrong': 34, 'all': 35, 'three': 36, 'of': 37, 'need': 38, 'to': 39, 'that': 40, 'are': 41, 'another': 42, 'chance': 43, 'both': 44, 'tom': 45, 'and': 46, 'mary': 47, 'work': 48, 'as': 49, 'models': 50, 'have': 51, 'minutes': 52, 'could': 53, 'close': 54, 'door': 55, 'plant': 56, 'pumpkins': 57, 'year': 58, 'ever': 59, 'study': 60, 'library': 61, 'don': 62, 'be': 63, 'deceived': 64, 'by': 65, 'appearances': 66, 'excuse': 67, 'speak': 68, 'english': 69, 'people': 70, 'true': 71, 'meaning': 72, 'germany': 73, 'produced': 74, 'many': 75, 'scientists': 76, 'guess': 77, 'whose': 78, 'birthday': 79, 'it': 80, 'today': 81, 'acted'

我们现在可以将原始英语句子转换为整数序列：

In [11]:
data_en = en_tokenizer.texts_to_sequences(raw_data_en) 

最后，我们需要填充零，以便所有序列具有相同的长度。 否则，我们以后将无法创建tf.data.Dataset对象。

In [12]:
data_en = tf.keras.preprocessing.sequence.pad_sequences(data_en,
                                                        padding='post')

In [58]:
print(data_en[:3])

[[ 8  5 21 22 23  0  0  0  0  0]
 [24 25  6 26 27 28  1  0  0  0]
 [ 5 29 30 31 32  9  8  7  6  1]]


继续用法语句子做同样的事情：

In [14]:
fr_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')

# ATTENTION: always finish with fit_on_texts before moving on
fr_tokenizer.fit_on_texts(raw_data_fr_in)
fr_tokenizer.fit_on_texts(raw_data_fr_out)

data_fr_in = fr_tokenizer.texts_to_sequences(raw_data_fr_in)
data_fr_in = tf.keras.preprocessing.sequence.pad_sequences(data_fr_in,
                                                           padding='post')

data_fr_out = fr_tokenizer.texts_to_sequences(raw_data_fr_out)
data_fr_out = tf.keras.preprocessing.sequence.pad_sequences(data_fr_out,
                                                            padding='post')

fit_on_text(texts) :使用一系列文档来生成token词典，texts为list类，每个元素为一个文档。

texts_to_sequences(texts):将多个文档转换为word下标的向量形式,shape为[len(texts)，len(text)] -- (文档数，每条文档的长度)

texts_to_matrix(texts): 将多个文档转换为矩阵表示,shape为[len(texts),num_words]

我们可以在不同的语料库上多次调用fit_on_texts，它会自动更新词汇表。 在使用texts_to_sequences之前，请务必先记得fit_on_texts。

最后一步很简单，我们只需要创建一个tf.data.Dataset的实例：

In [47]:
BATCH_SIZE = 5

dataset = tf.data.Dataset.from_tensor_slices(
    (data_en, data_fr_in, data_fr_out))
dataset = dataset.shuffle(20).batch(BATCH_SIZE)

### 没有注意力机制的Seq2Seq

但现在，我们可能已经知道注意机制是机器翻译任务的“标准配置”。 但我首先实现没有注意力机制的Seq2Seq作为baseline。

实验过程中，我们会感受到：

使用最新的TensorFlow2.0的tf.keras非常简单
能够回答：为什么需要注意力机制？

我们将从编码器开始。 在编码器内部，存在嵌入层和RNN层（可以是简单RNN或LSTM或GRU）实验中我们使用的是LSTM。 在每个前向传递中，它接收一批序列和初始状态并返回输出序列以及最终状态：

In [48]:
class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, lstm_size):
        super(Encoder, self).__init__()
        self.lstm_size = lstm_size
        # Embedding Layer
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        # Encode LSTM Layer
        self.lstm = tf.keras.layers.LSTM(
            lstm_size, return_sequences=True, return_state=True)

    def call(self, sequence, states):
        embed = self.embedding(sequence)
        output, state_h, state_c = self.lstm(embed, initial_state=states)

        return output, state_h, state_c

    def init_states(self, batch_size):
        return (tf.zeros([batch_size, self.lstm_size]),
                tf.zeros([batch_size, self.lstm_size]))

我们已经完成了编码器。 接下来，让我们创建解码器。 没有注意机制的情况下，解码器基本上与编码器相同，只是它有一个Dense层将RNN的输出映射到词汇空间：

In [49]:
class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, lstm_size):
        super(Decoder, self).__init__()
        self.lstm_size = lstm_size
        # Embedding Layer
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        # Decode LSTM Layer
        self.lstm = tf.keras.layers.LSTM(
            lstm_size, return_sequences=True, return_state=True)
        self.dense = tf.keras.layers.Dense(vocab_size)

    def call(self, sequence, state):
        embed = self.embedding(sequence)
        lstm_out, state_h, state_c = self.lstm(embed, state)
        logits = self.dense(lstm_out)

        return logits, state_h, state_c

    def init_states(self, batch_size):
        return (tf.zeros([batch_size, self.lstm_size]),
                tf.zeros([batch_size, self.lstm_size]))

编码器的最终状态将充当解码器的初始状态。 这是Seq2Seq模型的语言模型和解码器之间的差异。这就是我们需要创建的解码器。 在继续之前，让我们检查一下我们是否没有犯任何错误：

In [50]:
EMBEDDING_SIZE = 32
LSTM_SIZE = 64

en_vocab_size = len(en_tokenizer.word_index) + 1
encoder = Encoder(en_vocab_size, EMBEDDING_SIZE, LSTM_SIZE)

fr_vocab_size = len(fr_tokenizer.word_index) + 1
decoder = Decoder(fr_vocab_size, EMBEDDING_SIZE, LSTM_SIZE)

source_input = tf.constant([[1, 3, 5, 7, 2, 0, 0, 0]])
initial_state = encoder.init_states(1)
encoder_output, en_state_h, en_state_c = encoder(source_input, initial_state)

target_input = tf.constant([[1, 4, 6, 9, 2, 0, 0]])
decoder_output, de_state_h, de_state_c = decoder(target_input, (en_state_h, en_state_c))

print('Source sequences', source_input.shape)
print('Encoder outputs', encoder_output.shape)
print('Encoder state_h', en_state_h.shape)
print('Encoder state_c', en_state_c.shape)

print('\nDestination vocab size', fr_vocab_size)
print('Destination sequences', target_input.shape)
print('Decoder outputs', decoder_output.shape)
print('Decoder state_h', de_state_h.shape)
print('Decoder state_c', de_state_c.shape)

'''
Source sequences (1, 8)
Encoder outputs (1, 8, 64)
Encoder state_h (1, 64)
Encoder state_c (1, 64)
Destination vocab size 107
Destination sequences (1, 7)
Decoder outputs (1, 7, 107)
Decoder state_h (1, 64)
Decoder state_c (1, 64)
'''

Source sequences (1, 8)
Encoder outputs (1, 8, 64)
Encoder state_h (1, 64)
Encoder state_c (1, 64)

Destination vocab size 110
Destination sequences (1, 7)
Decoder outputs (1, 7, 110)
Decoder state_h (1, 64)
Decoder state_c (1, 64)


'\nSource sequences (1, 8)\nEncoder outputs (1, 8, 64)\nEncoder state_h (1, 64)\nEncoder state_c (1, 64)\nDestination vocab size 107\nDestination sequences (1, 7)\nDecoder outputs (1, 7, 107)\nDecoder state_h (1, 64)\nDecoder state_c (1, 64)\n'

接下来要做的是定义一个损失函数。 由于我们将零填充到序列中，因此在计算损失时不要考虑这些零：

In [51]:
def loss_func(targets, logits):
    crossentropy = tf.keras.losses.SparseCategoricalCrossentropy(
        from_logits=True)
    mask = tf.math.logical_not(tf.math.equal(targets, 0))
    mask = tf.cast(mask, dtype=tf.int64)
    loss = crossentropy(targets, logits, sample_weight=mask)

    return loss

创建优化器

In [52]:
optimizer = tf.keras.optimizers.Adam()

在创建训练循环之前，让我们定义一个用于推理目的的方法。 它的作用基本上是前向传递，但是我们将输入<start>标识，而不是目标序列。 每个下一个时间步都将最后一个时间步的输出作为输入，直到我们点击<end>标记或输出序列超过特定长度：

In [53]:
def predict(test_source_text=None):
    if test_source_text is None:
        test_source_text = raw_data_en[np.random.choice(len(raw_data_en))]
    print(test_source_text)
    test_source_seq = en_tokenizer.texts_to_sequences([test_source_text])
    print(test_source_seq)

    en_initial_states = encoder.init_states(1)
    en_outputs = encoder(tf.constant(test_source_seq), en_initial_states)

    de_input = tf.constant([[fr_tokenizer.word_index['<start>']]])
    de_state_h, de_state_c = en_outputs[1:]
    out_words = []

    while True:
        de_output, de_state_h, de_state_c = decoder(
            de_input, (de_state_h, de_state_c))
        de_input = tf.argmax(de_output, -1)
        out_words.append(fr_tokenizer.index_word[de_input.numpy()[0][0]])

        if out_words[-1] == '<end>' or len(out_words) >= 20:
            break

    print(' '.join(out_words))


现在我们已经准备好创建训练函数，我们执行前向传递，然后是反向传播。 有两件事需要记住：

我们使用@tf.function装饰器来推进静态图形计算（当你想调试时删除它）

计算需要放在tf.GradientTape（）下以跟踪Gradient

In [54]:
@tf.function
def train_step(source_seq, target_seq_in, target_seq_out, en_initial_states):
    with tf.GradientTape() as tape:
        en_outputs = encoder(source_seq, en_initial_states)
        en_states = en_outputs[1:]
        de_states = en_states

        de_outputs = decoder(target_seq_in, de_states)
        logits = de_outputs[0]
        loss = loss_func(target_seq_out, logits)

    variables = encoder.trainable_variables + decoder.trainable_variables
    gradients = tape.gradient(loss, variables)
    optimizer.apply_gradients(zip(gradients, variables))

    return loss

最后，这里是训练循环。 在每个训练批次，我们都会抓取批量数据还打印出损失值并查看模型在每个批次结束时的表现

In [56]:
NUM_EPOCHS = 300

for e in range(NUM_EPOCHS):
    en_initial_states = encoder.init_states(BATCH_SIZE)
    

    for batch, (source_seq, target_seq_in, target_seq_out) in enumerate(dataset.take(-1)):
        loss = train_step(source_seq, target_seq_in,
                          target_seq_out, en_initial_states)

    print('Epoch {} Loss {:.4f}'.format(e + 1, loss.numpy()))
    
    try:
        predict()
    except Exception:
      continue

test_sents = (
    'What a ridiculous concept!',
    'Your idea is not entirely crazy.',
    "A man's worth lies in what he is.",
    'What he did is very wrong.',
    "All three of you need to do that.",
    "Are you giving me another chance?",
    "Both Tom and Mary work as models.",
    "Can I have a few minutes, please?",
    "Could you close the door, please?",
    "Did you plant pumpkins this year?",
    "Do you ever study in the library?",
    "Don't be deceived by appearances.",
    "Excuse me. Can you speak English?",
    "Few people know the true meaning.",
    "Germany produced many scientists.",
    "Guess whose birthday it is today.",
    "He acted like he owned the place.",
    "Honesty will pay in the long run.",
    "How do we know this isn't a trap?",
    "I can't believe you're giving up.",
)

for test_sent in test_sents:
    test_sequence = normalize_string(test_sent)
    predict(test_sequence)

Epoch 1 Loss 3.4896
Did you plant pumpkins this year ?
[[13, 2, 56, 57, 19, 58, 3]]
est <end>
Epoch 2 Loss 3.6161
Did you plant pumpkins this year ?
[[13, 2, 56, 57, 19, 58, 3]]
vous <end>
Epoch 3 Loss 3.7476
A man s worth lies in what he is .
[[5, 29, 30, 31, 32, 9, 8, 7, 6, 1]]
vous <end>
Epoch 4 Loss 3.3962
Both Tom and Mary work as models .
[[44, 45, 46, 47, 48, 49, 50, 1]]
a <end>
Epoch 5 Loss 3.6558
Honesty will pay in the long run .
[[86, 87, 88, 9, 4, 89, 90, 1]]
vous <end>
Epoch 6 Loss 3.6598
What a ridiculous concept !
[[8, 5, 21, 22, 23]]
vous <end>
Epoch 7 Loss 3.2174
Guess whose birthday it is today .
[[77, 78, 79, 80, 6, 81, 1]]
<end>
Epoch 8 Loss 3.0991
Few people know the true meaning .
[[17, 70, 20, 4, 71, 72, 1]]
<end>
Epoch 9 Loss 3.0893
What he did is very wrong .
[[8, 7, 13, 6, 33, 34, 1]]
<end>
Epoch 10 Loss 3.0291
Germany produced many scientists .
[[73, 74, 75, 76, 1]]
vous <end>
Epoch 11 Loss 3.1738
A man s worth lies in what he is .
[[5, 29, 30, 31, 32, 9, 8, 

Epoch 78 Loss 1.6348
A man s worth lies in what he is .
[[5, 29, 30, 31, 32, 9, 8, 7, 6, 1]]
<end>
Epoch 79 Loss 1.5762
How do we know this isn t a trap ?
[[91, 10, 92, 20, 19, 93, 12, 5, 94, 3]]
vous vous vous vous porte s ? <end>
Epoch 80 Loss 1.2382
Are you giving me another chance ?
[[41, 2, 14, 15, 42, 43, 3]]
vous vous une ? <end>
Epoch 81 Loss 1.2653
How do we know this isn t a trap ?
[[91, 10, 92, 20, 19, 93, 12, 5, 94, 3]]
vous vous vous vous vous ? <end>
Epoch 82 Loss 1.4140
Both Tom and Mary work as models .
[[44, 45, 46, 47, 48, 49, 50, 1]]
tom et travaillent deux deux deux . <end>
Epoch 83 Loss 1.4398
All three of you need to do that .
[[35, 36, 37, 2, 38, 39, 10, 40, 1]]
vous vous vous autre ? <end>
Epoch 84 Loss 1.3642
Guess whose birthday it is today .
[[77, 78, 79, 80, 6, 81, 1]]
devine qui est anniversaire anniversaire aujourd hui ! <end>
Epoch 85 Loss 1.3873
He acted like he owned the place .
[[7, 82, 83, 7, 84, 4, 85, 1]]
il il est s s il possedait . <end>
Epoch 86 

Epoch 148 Loss 0.5110
Did you plant pumpkins this year ?
[[13, 2, 56, 57, 19, 58, 3]]
je vous prie de excuser ! <end>
Epoch 149 Loss 0.4430
What he did is very wrong .
[[8, 7, 13, 6, 33, 34, 1]]
ce il a fait tres . <end>
Epoch 150 Loss 0.5513
He acted like he owned the place .
[[7, 82, 83, 7, 84, 4, 85, 1]]
il s est comporte comme s il l . <end>
Epoch 151 Loss 0.4514
He acted like he owned the place .
[[7, 82, 83, 7, 84, 4, 85, 1]]
il s est comporte comme s il l . <end>
Epoch 152 Loss 0.4456
Are you giving me another chance ?
[[41, 2, 14, 15, 42, 43, 3]]
vous vous une chance ? <end>
Epoch 153 Loss 0.4984
What he did is very wrong .
[[8, 7, 13, 6, 33, 34, 1]]
ce il a fait tres . <end>
Epoch 154 Loss 0.5506
Your idea is not entirely crazy .
[[24, 25, 6, 26, 27, 28, 1]]
votre n pas pas folle . <end>
Epoch 155 Loss 0.4083
Don t be deceived by appearances .
[[62, 12, 63, 64, 65, 66, 1]]
vous laissez pas par les . <end>
Epoch 156 Loss 0.4568
How do we know this isn t a trap ?
[[91, 10, 92, 2

Epoch 213 Loss 0.1671
A man s worth lies in what he is .
[[5, 29, 30, 31, 32, 9, 8, 7, 6, 1]]
la valeur d un homme reside dans ce qu il est . <end>
Epoch 214 Loss 0.1740
Honesty will pay in the long run .
[[86, 87, 88, 9, 4, 89, 90, 1]]
l paye a la longue . <end>
Epoch 215 Loss 0.1524
Germany produced many scientists .
[[73, 74, 75, 76, 1]]
l a la longue . <end>
Epoch 216 Loss 0.2355
All three of you need to do that .
[[35, 36, 37, 2, 38, 39, 10, 40, 1]]
vous avez besoin de faire cela tous les trois . <end>
Epoch 217 Loss 0.1970
Can I have a few minutes please ?
[[11, 16, 51, 5, 17, 52, 18, 3]]
puis je avoir quelques minutes je vous prie ? <end>
Epoch 218 Loss 0.1442
Honesty will pay in the long run .
[[86, 87, 88, 9, 4, 89, 90, 1]]
l paye a la longue . <end>
Epoch 219 Loss 0.1944
Your idea is not entirely crazy .
[[24, 25, 6, 26, 27, 28, 1]]
votre n est pas completement . <end>
Epoch 220 Loss 0.1568
How do we know this isn t a trap ?
[[91, 10, 92, 20, 19, 93, 12, 5, 94, 3]]
comment sa

l paye a la longue . <end>
Epoch 277 Loss 0.1144
Your idea is not entirely crazy .
[[24, 25, 6, 26, 27, 28, 1]]
votre n est pas completement folle . <end>
Epoch 278 Loss 0.0685
What he did is very wrong .
[[8, 7, 13, 6, 33, 34, 1]]
ce qu il est tres mal . <end>
Epoch 279 Loss 0.0843
Are you giving me another chance ?
[[41, 2, 14, 15, 42, 43, 3]]
me donnez vous une chance ? <end>
Epoch 280 Loss 0.0725
Both Tom and Mary work as models .
[[44, 45, 46, 47, 48, 49, 50, 1]]
tom et mary travaillent tous les deux comme mannequins . <end>
Epoch 281 Loss 0.0735
He acted like he owned the place .
[[7, 82, 83, 7, 84, 4, 85, 1]]
il s est comporte comme s il possedait l endroit . <end>
Epoch 282 Loss 0.0891
Excuse me . Can you speak English ?
[[67, 15, 1, 11, 2, 68, 69, 3]]
je vous prie de m excuser ! savez vous parler anglais ? <end>
Epoch 283 Loss 0.0663
Do you ever study in the library ?
[[10, 2, 59, 60, 9, 4, 61, 3]]
est ce que vous etudiez a la bibliotheque des fois ? <end>
Epoch 284 Loss 0.084